In [ ]:
import MySQLdb
import pandas as pd
import tweepy
from tqdm.auto import tqdm
from envs import *

In [ ]:
con = MySQLdb.connect(user=db_username, passwd=db_pass,
                               host=db_host, db="mysql", use_unicode=True, charset="utf8mb4")
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
def getTweet(user_id, api, since_id):
    tweet_ids=[]
    created_ats=[]
    texts=[]
    tweet_jsons=[]
    for tweet in tweepy.Cursor(api.user_timeline, since_id=since_id, id=user_id).items():
            tweet_id = tweet.id
            created_at = tweet.created_at
            text = tweet.text
            tweet_json = json.dumps(tweet._json, ensure_ascii=False)
            tweet_ids.append(tweet_id)
            created_ats.append(created_at)
            texts.append(text)
            tweet_jsons.append(tweet_json)
    df = pd.DataFrame()
    df["tweet_id"] = tweet_ids
    df["created_at"] = created_ats
    df["text"]=texts
    df["user_id"]=user_id
    df["tweet_json"]=tweet_jsons
    return df

In [ ]:
def getDeleteTweet(user_id, con, api):
    user = api.get_user(user_id)
    tl_all = pd.read_sql("SELECT * FROM d2.tweet WHERE user_id={0}".format(user.id),  con=con)
    if tl_all.shape[0]==0:
        return tl_all
    since_id = tl_all.iloc[0].tweet_id-1
    tl_now = getTweet(user.id, api, since_id=since_id)
    tweet_ids = tl_now["tweet_id"].values
    deleted = tl_all["tweet_id"].apply(lambda w: w in tweet_ids)
    return tl_all[deleted==False]


In [ ]:
for t in getDeleteTweet("rikuta0811_dash",con,api).text:
    print(t)
    print()